In [6]:
import json
import re
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin

In [7]:
!python -m spacy download en_core_web_sm

  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [8]:
# Carrega modelo spaCy
nlp = spacy.load("en_core_web_sm")

In [9]:
# Limpeza e tokenização com spaCy
class TextCleaner(BaseEstimator, TransformerMixin):
    def clean_text(self, text):
        text = text.lower()
        text = re.sub(r"http\S+", "", text)
        text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
        text = re.sub(r"\s+", " ", text)

        doc = nlp(text)
        tokens = [
            token.lemma_ for token in doc
            if not token.is_stop and not token.is_punct and token.is_alpha
        ]
        return " ".join(tokens)

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [self.clean_text(text) for text in X]

In [10]:
# Classificação por regra simples
def rule_based_classification(text):
    text = text.lower()
    if "abuse" in text or "malicious" in text:
        return "Malicioso"
    elif "registro" in text or "whois" in text:
        return "Possivelmente benigno"
    else:
        return "Desconhecido"

In [11]:
# Modelo ML
def train_ml_model():
    texts = [
        "Malicious IP detected with 1/94 ratio",
        "No malicious activity found",
        "200 resolution(s) reported",
        "Benign behavior observed",
        "VT flagged this IP as malicious",
        "Clean IP, no issues detected",
    ]
    labels = [1, 0, 1, 0, 1, 0]

    pipeline = Pipeline([
        ('cleaner', TextCleaner()),
        ('tfidf', TfidfVectorizer()),
        ('clf', LogisticRegression())
    ])
    pipeline.fit(texts, labels)
    return pipeline

In [12]:
# Extração dos níveis e valores das taxonomias
def extract_taxonomy_info(taxonomies):
    levels = []
    values = []
    for tax in taxonomies:
        if 'level' in tax:
            levels.append(tax['level'])
        if 'value' in tax:
            values.append(tax['value'])
    return levels, values

In [13]:
# Geração de comentário com base em regras e estatísticas
def generate_comment(text, rule_label, stats):
    comment = f"Texto analisado: {text[:100]}... "
    comment += f"Classificação inicial: {rule_label}. "
    if stats:
        positives = stats.get("malicious", 0)
        total = sum(stats.values())
        comment += f"Detectado por {positives}/{total} mecanismos. "
    return comment

In [14]:
# Função principal
def main_from_json(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    # Extrai partes específicas do relatório
    whois_text = data["full"]["attributes"].get("whois", "")
    stats = data["full"]["attributes"].get("last_analysis_stats", {})
    taxonomies = data.get("summary", {}).get("taxonomies", [])

    # Classificação por regra
    rule_label = rule_based_classification(whois_text)

    # Classificação por modelo
    ml_pipeline = train_ml_model()
    ml_label = ml_pipeline.predict([whois_text])[0]

    # Processa taxonomias
    taxonomy_levels, taxonomy_values = extract_taxonomy_info(taxonomies)

    # Geração de comentário final
    comment = generate_comment(whois_text, rule_label, stats)
    if taxonomy_levels:
        comment += f" Classificação do relatório: {', '.join(taxonomy_levels)}."
    if taxonomy_values:
        comment += f" Detalhes adicionais: {', '.join(taxonomy_values)}."

    # Resultados
    print("Classificação por Regras:", rule_label)
    print("Classificação por ML:", "Malicioso" if ml_label == 1 else "Benigno")
    print("Níveis da Taxonomia:", taxonomy_levels)
    print("Comentário Gerado:", comment)

In [15]:
main_from_json("./assets/report.txt")

Classificação por Regras: Malicioso
Classificação por ML: Benigno
Níveis da Taxonomia: ['malicious', 'malicious']
Comentário Gerado: Texto analisado: NetRange: 104.16.0.0 - 104.31.255.255
CIDR: 104.16.0.0/12
NetName: CLOUDFLARENET
NetHandle: NET-104-... Classificação inicial: Malicioso. Detectado por 1/94 mecanismos.  Classificação do relatório: malicious, malicious. Detalhes adicionais: 1/94, 200 resolution(s).
